In [1]:
import numpy as np
from pathlib import Path
import re
import h5py

import matplotlib.pyplot as plt
from scipy.interpolate import griddata
%matplotlib widget
import matplotlib
matplotlib.use("Agg")


In [2]:
alle = {}
for f in Path("sirak/").glob("export_*ec.csv"):
    print(f.stem)
    alle[float(re.search(r"(1[45]\.[0-9])",f.stem)[0])] = np.loadtxt(f,delimiter=",",skiprows=5)

export_14.8sec
export_14.6Sec
export_14.4Sec
export_14.2Sec
export_15.0sec
export_14.0Sec


In [4]:
tider = list(alle.keys())
tider.sort()
xy = alle[tider[0]][:,0:2]
        
for t in tider:
    alle[t] = alle[t][:,3:]
uvw = np.zeros((6,26368,3))
for i,t in enumerate(tider):
    uvw[i] = alle[t]
x,y = np.mgrid[-1:1.11:1920j,-0.055:.055:300j]

In [26]:
xy = alle[:,0:2]
        
uvw = alle[:,3:]


In [10]:
uvw[0].shape

(26368, 3)

In [23]:
resampled=griddata((xy), np.linalg.norm((uvw[3]-uvw[2]),axis=1),(x,y), method='nearest')
fig,ax = plt.subplots(figsize=(1000/myDPI,100/myDPI),dpi=myDPI)
ax.imshow(resampled.T[:,1500:],extent=(-1,1.11,-0.055,0.055),origin='lower',aspect='auto')
plt.show()
fig.savefig(f"sirak/diff.png")
plt.close()

In [9]:
myDPI  =150
length = 2.11
pts = length * 910
for i in range(6):
    resampled=griddata((xy), np.linalg.norm(uvw[i],axis=1),(x,y), method='nearest')
    fig,ax = plt.subplots(figsize=(1000/myDPI,100/myDPI),dpi=myDPI)
    ax.imshow(resampled.T,extent=(-1,1.11,-0.055,0.055),origin='lower')
    # ax.imshow(uvw,extent=(-1,1.2,-.06,.06),origin='lower')
    plt.show()
    fig.savefig(f"sirak/sirak{i}.png")
    plt.close()

In [7]:
path = "chirag.hdf5"
print(f"Skal lagra i {path}")
with h5py.File(path, 'w') as f:
    f.create_dataset("x", data=xy, compression="gzip", compression_opts=9)
    f.create_dataset("U", data=uvw, compression="gzip", compression_opts=9)
    f.create_dataset("t", data=np.asarray(tider), compression="gzip", compression_opts=9)
    f.attrs.create('I', data=I)
    f.attrs.create('J', data=J)
    

Skal lagra i chirag.hdf5


In [ ]:
utfilnamn=Path("chirag.mp4")
slow = 1
fps = 60
t_min = tider[0]
t_max = tider[-1]
steps = (t_max - t_min) * fps


# t_list = np.arange(t_min*fps,t_max*fps)/fps
t_list = np.arange(t_min*fps,t_max*fps)/fps
t_list_part = np.arange(t_min*20,t_max*20)/20

# piv_range = ranges()
hdf5_fil = Path("chirag.hdf5")
with h5py.File(hdf5_fil, 'r') as dataset:
    (I,J)=dataset.attrs['I'],dataset.attrs['J']

    Umx = np.array(dataset['Umx'])[t_min*20:t_max*20,:]
    Umx_reshape = Umx.reshape((len(Umx),J,I))#[:,piv_range[0],piv_range[1]]
    Vmx = np.array(dataset['Vmx'])[t_min*20:t_max*20,:]
    Vmx_reshape = Vmx.reshape((len(Vmx),J,I))#[:,piv_range[0],piv_range[1]]

    # ribs = np.array(dataset['ribs'])
    
    x = np.array(dataset['x'])
    y = np.array(dataset['y'])

x_reshape = x.reshape(J,I)#[piv_range]
y_reshape = y.reshape(J,I)#[piv_range]
        
V_mag_reshape = np.hypot(Umx_reshape, Vmx_reshape)        
# V_mag_reshape = np.hypot(U[2], U[3])
V_mag_interp = interp1d(range(t_min*fps,t_max*fps,int(fps/20)), V_mag_reshape, axis=0)

myDPI = 300

fig, ax = plt.subplots(figsize=(1234/myDPI,1080/myDPI),dpi=myDPI)

field = ax.imshow(V_mag_reshape[0,:,:], extent=[x_reshape[0,0],x_reshape[0,-1], y_reshape[-1,0], y_reshape[0,0]], interpolation='none')
    
ax.set_xlim([x_reshape[0,0],x_reshape[0,-1]])
draw_rect(ax, ribs)

filmstart = [datetime.datetime.now()]

def nypkt(i):
    field.set_data(V_mag_interp(i))
    return 

#ax.axis('equal')
# ani = animation.FuncAnimation(fig, nypkt, frames=np.arange(1,steps),interval=50)
FFwriter = animation.FFMpegWriter(fps=fps)
ani = animation.FuncAnimation(fig, nypkt, frames=np.arange(0,steps-int(fps/20)),interval=(slow*1000/(fps)), blit=False)
# plt.show()

starttid = datetime.datetime.now()
ani.save(utfilnamn, writer=FFwriter)
print(f"Brukte {datetime.datetime.now()-starttid} på å lagra filmen")
plt.close()
